<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/data-augmentation/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 as cv
import os
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/"

ModuleNotFoundError: No module named 'google.colab'

In [38]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\FER2013-Combined')
LABELS = os.listdir(PATH)

In [20]:
count = []
for label in LABELS:
  dir = f'{PATH}/{label}'
  folder_item_count = len(os.listdir(dir))
  count.append(folder_item_count)
  print(f'{label}: {folder_item_count}')

HIGHEST_COUNT = min(count)
print(f'Highest count: {HIGHEST_COUNT}')

angry: 3000
disgust: 547
fear: 3000
happy: 3000
neutral: 3000
sad: 3000
surprise: 3000
Highest count: 547


In [39]:
for i, label in enumerate(LABELS):
  path = f'{PATH}/{label}'
  folder_path = f'C:\\Users\\vinar\\Downloads\\FER2013-down-sampling\\{label}'
  os.makedirs(folder_path, exist_ok=True)

  for j, img_path in enumerate(os.listdir(path)):
    if(j == 3000):
      break
    img = cv.imread(f'{path}/{img_path}')
    new_path = f'{folder_path}\\{img_path}'
    cv.imwrite(new_path, img)

In [4]:
def rotate(img):
  Cx, Cy = img.shape[:2]
  rand_angle = random.randint(-180,180)
  M = cv.getRotationMatrix2D((Cy//2, Cx//2),rand_angle ,1)
  img = cv.warpAffine(img, M, (Cx, Cy))
  return img

def flip(img):
  num = [0, 1, -1]
  img = cv.flip(img, random.choice(num))
  return img

def noise(img):
  row, col = img.shape[:2]
  number_of_pixels = random.randint(300, 1000)
  for i in range(number_of_pixels):
    y_coord=random.randint(0, row - 1)
    x_coord=random.randint(0, col - 1)
    img[y_coord][x_coord] = 255

  number_of_pixels = random.randint(300 , 1000)
  for i in range(number_of_pixels):
      y_coord=random.randint(0, row - 1)
      x_coord=random.randint(0, col - 1)
      img[y_coord][x_coord] = 0
  return img

In [5]:
data = {}

for i, label in enumerate(LABELS):
  path = f'{PATH}/{label}'
  data[label] = []
  for j, img_path in enumerate(os.listdir(path)):
    img = cv.imread(f'{path}/{img_path}')
    data[label].append(img)

In [9]:
for label in LABELS:
  images = data[label]

  path = f'C:\\Users\\vinar\\Downloads\\facial-emotion-recognition-augmented\\{label}'
  os.makedirs(path, exist_ok=True)

  for j, img in enumerate(images):
    if(len(os.listdir(path)) >= HIGHEST_COUNT):
      break

    num = [1,2,3]
    rand = random.choice(num)

    for i in range(8):
      if(len(os.listdir(path)) >= HIGHEST_COUNT):
        break
      if(rand == 1):
        img = rotate(img)
      elif(rand == 2):
        img = flip(img)
      elif(rand == 3):
        img = noise(img)


      img_name = f'{path}/{label}_{j}_{i}.png'
      cv.imwrite(img_name, img)

In [3]:
count = []
for label in LABELS:
  dir = f'{PATH}/{label}'
  folder_item_count = len(os.listdir(dir))
  count.append(folder_item_count)
  print(f'{label}: {folder_item_count}')

LOWEST_COUNT = min(count)
print(f'Highest count: {LOWEST_COUNT}')

angry: 890
disgust: 439
fear: 570
happy: 1406
neutral: 524
sad: 746
surprise: 775
Highest count: 439


In [6]:
for label in LABELS:
  folder_dir = f'{PATH}/{label}'

  path = f'C:\\Users\\vinar\\Downloads\\down-sampling\\{label}'
  os.makedirs(path, exist_ok=True)

  for i, image in enumerate(os.listdir(folder_dir)):
    if(i == LOWEST_COUNT):
      break

    img = cv.imread(f'{folder_dir}/{image}')
    cv.imwrite(f'{path}/{image}', img)

In [24]:
import os
import cv2
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations import HorizontalFlip, RandomBrightnessContrast, ShiftScaleRotate
from albumentations import Normalize
from tqdm import tqdm
import numpy as np

In [45]:
transform = A.Compose([
    A.Resize(224, 224),
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 5), p=0.5),
        A.GaussNoise(var_limit=(10, 30), p=0.5)
    ])
])

In [43]:
def augment_and_save(input_dir, output_dir, num_augments=3):
    """
    Creates augmented images from a dataset and saves them to disk.

    Args:
        input_dir (str): Path to the original dataset directory (with class subfolders).
        output_dir (str): Path to save the augmented dataset.
        num_augments (int): Number of augmented images to create per original image.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through each class folder
    for class_name in os.listdir(input_dir):
        # if(class_name != 'disgust'):
        #     continue
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder files

        # Create a corresponding folder in the output directory
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        # Loop through each image in the class folder
        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            # if(len(os.listdir(f'{output_dir}/{class_name}')) == 3000):
            #     break
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)  # Load the image
            if image is None:
                continue  # Skip unreadable images
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Save original image
            original_output_path = os.path.join(output_class_path, img_name)
            cv2.imwrite(original_output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Generate and save augmented images
            for i in range(num_augments):
                # if(len(os.listdir(f'{output_dir}/{class_name}')) == 3000):
                #     break
                augmented = transform(image=image)['image']  # Apply augmentation
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
                aug_img_path = os.path.join(output_class_path, aug_img_name)
                cv2.imwrite(aug_img_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

In [44]:
input_dir = "C:\\Users\\vinar\\Downloads\\FER2013-down-sampling"
output_dir = "C:\\Users\\vinar\\Downloads\\FER2013-augmented"
augment_and_save(input_dir, output_dir, num_augments=3)

Processing surprise: 100%|██████████| 3000/3000 [00:42<00:00, 69.94it/s]
